In [6]:
import psycopg2
import pandas as pd
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU, Input
from keras.models import Sequential
from pathlib import Path
import os
import pickle as p
import tensorflow as tensorflow
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from keras.preprocessing.text import Tokenizer as token
from tensorflow.keras.preprocessing.sequence import pad_sequences
EMBEDDING_VECTOR_LENGTH = 33

In [7]:
class Tokenizer:
    TOP_K = 20000
    MAX_SEQUENCE_LENGTH = 33

    def __init__(self, train_texts):
        # токенизатор собственно
        #
        self.train_texts = train_texts
        self.tokenizer = token(num_words=self.TOP_K)

    def train_tokenize(self):
        #
        #
        max_length = len(max(self.train_texts, key=len))
        self.max_length = min(max_length, self.MAX_SEQUENCE_LENGTH)
        self.tokenizer.fit_on_texts(self.train_texts)

    def vectorize_input(self, tweets):
        #
        #
        tweets = self.tokenizer.texts_to_sequences(tweets)
        tweets = pad_sequences(tweets, maxlen=self.max_length, truncating='post', padding='post')
        return tweets

In [8]:
from nltk.corpus import stopwords
from string import punctuation
from pymystem3 import Mystem
import re
## данные в базе леманатизированы
class CommonPreprocessing:

    @classmethod
    def preprocess_text(cls, text):
        # предобработчик приводит слова в начальную форму. Данные в базе находятся в леманатированном состоянии перед подачей на токенизацию в предикт слово тоже леманатируется
        #
        try:
            tokens = str(text)
            tokens = Mystem().lemmatize(text.lower())
            tokens = [token for token in tokens if token not in stopwords.words('russian')
                      and token != ' '
                      and token.strip() not in punctuation]
            tokens = [
                token for token in tokens if token not in stopwords.words('english')]

            text = ' '.join(tokens).rstrip('\n')
            pattern3 = r'[\d]'
            pattern2 = '[.]'
            text = re.sub(pattern3, '', text)
            text = re.sub(pattern2, '', text)
            text = re.sub('  ', ' ', text)
            return text
        except:
            return 'The exception is in CommonPreprocessing.preprocess_text'

In [9]:
def predict(inpt, tmap, model, tokenizer):
    #
    #
    model = load_model(model)
    inn = []
    pr = CommonPreprocessing()
    for i in inpt:
        inn.append(pr.preprocess_text(i))

    with open(tokenizer, 'rb') as handle:
        tokenizer = p.load(handle)

    tokenized_inpt = tokenizer.vectorize_input(inn)
    scoreplu = model.predict(tokenized_inpt)
    outpt = tmap[scoreplu.argmax(axis=-1)[0]]
    return outpt

In [13]:
EMOTIONSMAPA = {0: '😞', 1: '🤬', 2: '😨', 3: '😊', 4: '❤', 5: '😳', 6: ''}
modelpath = next(Path().rglob('0_lstmemotionsmodel.h5'))
tokenizerpath = next(Path().rglob('0_lstmemotionstokenizer.pickle'))
emotion = predict("это семпл из трейна нужно предикт на тексте что не в трейне делать и желательно на множестве тк там всё оносительно по статистике",EMOTIONSMAPA, 'Models/0_lstmemotionsmodel.h5', 'Tokenizers/0_lstmemotionstokenizer.pickle')
emotion

5/5 [==============================] - 1s 78ms/step


'😊'